# Compare Manual to Automatic Meander Characterization

In this code we will compare the manual to the automatic detection


## Start Script
### Importing Packages

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [2]:
# Importing Packages
import os
import time
import copy
import numpy as np
import pyproj
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from IPython.display import Image, display, HTML
from joblib import Parallel, delayed
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display
from scipy.spatial import Delaunay
from scipy.spatial.distance import euclidean

# Importing pyMeander Packages
from pyMeander import Rivers
from pyMeander import RiverFunctions as RF
from pyMeander import ExtractNHDPlusHRData as NHD
from pyMeander.utilities import utilities as utl
from pyMeander.utilities import filesManagement as FM
from pyMeander import Logger
from pyMeander import WaveletTreeFunctions as WTFunc

In [3]:
# Start Logger
logger = Logger(console=True)

2023-08-16 10:42:21,384[INFO] set_logger: Starting log


In [4]:
# Parameters
path_projects = './projects/old_characterization/rivers_manual/'
projects = utl.get_folders(path_projects)
path_projects_all = [f'{path_projects}{p}/' for p in projects]
path_projects_c = './projects/old_characterization/corrected/'
path_projects_out = [f'{path_projects_c}{p}/' for p in projects]

print(projects)

# Current projection of information
projection = 'esri:102003'

['0104', '0106', '0107', '0307', '0406', '0408', '0412', '0504', '0510', '0601', '0701', '0708', '0714', '0803', '0808', '0904', '1002', '1006', '1009', '1017', '1021', '1030', '1108', '1110', '1113', '1202', '1206', '1208', '1211', '1305', '1307', '1402', '1404', '1406', '1701', '1703', '1704', '1801', '1805', '1807']


### Look for inconsistencies to erase

In [135]:
# Select project to load
project = '1701'
try:
    i_p = projects.index(project)
except ValueError:
    raise ValueError(f'Project "{project}" not found')
project_to_load = path_projects_all[i_p]
print('Set project to load: ' + project_to_load)

Set project to load: ./projects/old_characterization/rivers_manual/1701/


In [136]:
# Start Rivers object
rivers = Rivers(logger=logger)
river_network_file = f'{project_to_load}/rivers.hdf5'
rivers.load_river_network(
    river_network_file,
    meanders_database_file=f'{project_to_load}/meander_database.csv')
id_rivers = rivers.id_values

# Select River
id_river = id_rivers[0]
river = rivers[id_river]


2023-08-14 12:12:15,650[INFO] info: Start Logger in Rivers
2023-08-14 12:12:15,677[WARNING] warning: Spline not calculated, calculating splines with `calculate_spline()`


In [ ]:
rivers[id_river].interactive_meander_characterization_plot()

In [139]:
# Check meanders that will later be removed
import dotenv
satellite_figure = True
dotenv.load_dotenv()
MAPBOX_TOKEN = os.environ['MAPBOX_TOKEN']
rivers[id_river].interactive_meander_characterization_plot(
    mapbox_token=MAPBOX_TOKEN, current_crs='esri:102003', zoom=7)


FigureWidget({
    'data': [{'lat': array([48.0792186 , 48.07905048, 48.0788875 , ..., 47.79516977, 47.79525421,
                            47.7953387 ]),
              'line': {'color': 'blue', 'width': 2},
              'lon': array([-117.20615154, -117.20616821, -117.20622389, ..., -117.53391959,
                            -117.53413134, -117.53434298]),
              'mode': 'lines',
              'name': 'River',
              'type': 'scattermapbox',
              'uid': 'd387687e-3841-4003-b96b-a4ec9ee85f4d'}],
    'layout': {'autosize': True,
               'height': 500,
               'hovermode': 'closest',
               'mapbox': {'accesstoken': ('pk.eyJ1IjoiZGdkMDQyIiwiYSI6ImN' ... '2ZzcifQ.Y0LQAws3OWtzte09c-zv0g'),
                          'bearing': 0,
                          'center': {'lat': 47.890408916359675, 'lon': -117.36476595925951},
                          'pitch': 0,
                          'style': 'satellite',
                          'zoom': 7},
 

In [138]:
# Remove meander
if project == '0104':
    meanders_to_remove = [
        17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 4, 5, 20, 22, 31, 32, 6, 7]
elif project == '0106':
    meanders_to_remove = [14, 15, 13, 12, 11, 10, 8, 9, 7, 3, 2, 17, 16]
elif project == '0107':
    meanders_to_remove = [
        52, 51, 49, 50, 48, 47, 45, 44, 46, 43, 42, 41, 38, 39, 40, 37, 35, 32,
        29, 27, 26, 21, 20, 24, 23, 22, 19, 18, 16, 17, 15, 13, 14, 12, 11, 10,
        7, 8, 9, 6, 5, 1, 3, 2, 36]
elif project == '0307':
    meanders_to_remove = [8, 6, 7, 5, 6, 5]
elif project == '0406':
    meanders_to_remove = np.arange(1, 90)
elif project == '0412':
    meanders_to_remove = [12, 13, 11, 10, 7, 8, 4, 5, 6]
elif project == '0701':
    meanders_to_remove = [
        27, 23, 22, 20, 21, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 7, 8, 6,
        5, 4]
elif project == '0714':
    meanders_to_remove = [9]
elif project == '0803':
    meanders_to_remove = [
        6, 3, 7, 8, 9, 12, 11, 10, 13, 14, 15, 17, 16, 18, 19, ]
elif project == '1002':
    meanders_to_remove = [9, 7, 8, 5, 6]
elif project == '1009':
    meanders_to_remove = [14, 12, 11, 10, 9, 8, 7, 6, 4]
elif project == '1030':
    meanders_to_remove = [8, 7, 11, 10, 12, 13, 16, 14, 17, 15, 18, 19]
elif project == '1108':
    meanders_to_remove = [
        21, 19, 18, 17, 16, 15, 14, 13, 9, 10, 11, 12, 7, 8, 6, 5, 4, ]
elif project == '1110':
    meanders_to_remove = [
        36, 35, 33, 34, 32, 31, 30, 29, 28, 27, 26, 25, 23, 24, 22, 21, 20,
        19, 18, 17, 16, 15, 14, 13, 12, 10, 9, 8, 7, 6, 4, 5, ]
elif project == '1113':
    meanders_to_remove = [19, 18, 17, 16, 15, 14, 13, 12, 11, 8, 7, 5, ]
elif project == '1202':
    meanders_to_remove = [
        26, 25, 23, 24, 27, 28, 30, 29, 33, 34, 35, 36, 37, 38, 39, 41, 41,
        43, 44, 45, 46, 47, 48, 49, 50, 52, 54, 51, 55, 53, 56, 58, 57, 61, 62,
        65, 64, 66, 21, 22, 20]
elif project == '1206':
    meanders_to_remove = [1, 2, 3, 4, 7, 8, 9, 11, ]
elif project == '1208':
    meanders_to_remove = [12, 15, 13, 11, 10, 9, 8, 7, 17, 18, 6, 5, ]
elif project == '1404':
    # meanders_to_remove = [12, 15, 13, 11, 10, 9, 8, 7, 17, 18, 6, 5, ]
    meanders_to_remove = np.arange(1, 120)
elif project == '1406':
    meanders_to_remove = np.arange(1, 91)
elif project == '1701':
    meanders_to_remove = np.arange(1, 34)
else:
    meanders_to_remove = []

for id_m in meanders_to_remove:
    river.remove_meander(id_m)


# Save new information

In [140]:
rivers.save_rivers(
    path_out=f'{path_projects_c}{project}/', file_name='rivers.hdf5',
    file_name_meander_database='meander_database.csv')

2023-08-14 12:13:37,771[INFO] info: No CWT poly or ZC lines extracted
2023-08-14 12:13:37,773[INFO] info: No tree scales extracted


## Work with New datasets

In [5]:
# Work with new dataset
# Select project to load
project = '1701'
try:
    i_p = projects.index(project)
except ValueError:
    raise ValueError(f'Project "{project}" not found')
project_to_load = path_projects_out[i_p]
print('Set project to load: ' + project_to_load)

# Start Rivers object
rivers = Rivers(logger=logger)
river_network_file = f'{project_to_load}/rivers.hdf5'
rivers.load_river_network(
    river_network_file,
    meanders_database_file=f'{project_to_load}/meander_database.csv')
id_rivers = rivers.id_values

# Select River
id_river = id_rivers[0]
river = rivers[id_river]


2023-08-16 10:42:29,082[INFO] info: Start Logger in Rivers
2023-08-16 10:42:29,133[WARNING] warning: Spline not calculated, calculating splines with `calculate_spline()`


Set project to load: ./projects/old_characterization/corrected/1701/


In [6]:
# Check meanders that will later be removed
import dotenv
satellite_figure = True
dotenv.load_dotenv()
MAPBOX_TOKEN = os.environ['MAPBOX_TOKEN']
rivers[id_river].interactive_meander_characterization_plot(
    mapbox_token=MAPBOX_TOKEN, current_crs='esri:102003', zoom=7)

FigureWidget({
    'data': [{'lat': array([48.0792186 , 48.07905048, 48.0788875 , ..., 47.79516977, 47.79525421,
                            47.7953387 ]),
              'line': {'color': 'blue', 'width': 2},
              'lon': array([-117.20615154, -117.20616821, -117.20622389, ..., -117.53391959,
                            -117.53413134, -117.53434298]),
              'mode': 'lines',
              'name': 'River',
              'type': 'scattermapbox',
              'uid': '240e0fa9-3ea3-4e1e-8ca8-73b7e361fddc'},
             {'hovertemplate': ('<b>Meander ID: 0</b><br>Lambda' ... 'nuosity: 2.18<br>Radius: 74.27'),
              'lat': [47.77772739409348],
              'lon': [-117.5275898240229],
              'marker': {'color': 'red', 'size': 10},
              'mode': 'markers',
              'name': 'Meander 0',
              'showlegend': False,
              'type': 'scattermapbox',
              'uid': 'f9c34b6a-922e-4c9f-a578-a1e252b00e86'},
             {'hovertemplate

In [107]:
# Remove possible errors
river.remove_meander(57)

In [ ]:
# Look at individual meanders
id_meander = 42
f = river.meanders[id_meander].plot_meander()

In [9]:
# Save Final characterization
rivers.save_rivers(
    path_out=f'{path_projects_c}{project}/', file_name='rivers.hdf5',
    file_name_meander_database='meander_database.csv')

2023-08-16 12:03:55,827[INFO] info: No CWT poly or ZC lines extracted
2023-08-16 12:03:55,828[INFO] info: No tree scales extracted
